In [1]:
import os
import sys
# import tensorflow_hub as hub
import pandas as pd
# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Import maskRCNN
sys.path.append(ROOT_DIR)
import geopandas as gpd
import DataProcessing as DP
import json
import skimage
from tqdm import tqdm
import rasterio
from itertools import product
from rasterio import windows
import DataProcessing as DP
from pathlib import Path
import rioxarray
import shapely
from rasterio.warp import calculate_default_transform, reproject, Resampling
import shutil
import json
import random
import numpy as np

In [2]:
DATA_DIR = os.path.join(ROOT_DIR, "DigitalGlobe_FineTune")
label_dir = os.path.join(DATA_DIR, "geo-labels")
image_dir = os.path.join(DATA_DIR, "images")
image_proj_dir = os.path.join(DATA_DIR, "images_wgs84")

Path(image_proj_dir).mkdir(exist_ok=True, parents=True)

# image_dir = os.path.join(ROOT_DIR, "datasets_SH_PAN", "images_int8")
out_label_dir = os.path.join(ROOT_DIR, "processing_data_DG")

In [3]:
def re_projection(intif, outtif, dst_crs=None):
    
    if dst_crs is None:
        
        dst_crs = 'EPSG:4326'
    
    src = rasterio.open(intif)
    transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    with rasterio.open(outtif, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)




In [4]:
# need reprojection to standardize the images
reprojection = False

while reprojection:
    reprojection = False
    allimages = [file for file in os.listdir(image_dir) if file.endswith(".tif")]
    
    for file in tqdm(allimages):
        infile = os.path.join(image_dir, file)
        outfile = os.path.join(image_proj_dir, file)
        re_projection(infile, outfile, dst_crs=None)

In [5]:
image_dict = dict()
label_dict = dict()

for img in os.listdir(image_proj_dir):
    if img.endswith(".tif"):
        image_name = os.path.splitext(img)[0]
        if image_name not in image_dict:
            image_dict[image_name] = os.path.join(image_proj_dir, img)

for label in os.listdir(label_dir):
    if label.endswith(".shp"):
        label_name = os.path.splitext(label)[0]

        if label_name not in label_dict:
            label_dict[label_name] = os.path.join(label_dir, label)


In [6]:


def generate_annotation(single_image_path, single_label_path, label):

    """
    :param single_image_path: image path
    :param polygeos: a list of shapely geometries
    :param label: object label
    :return: save the generated annotation files to annotations folder
    """

    try:
        img = rasterio.open(single_image_path)
        width = img.width
        height = img.height
    except:
        print(img)
        raise


    filename = ".".join(os.path.basename(single_label_path).split('.')[:-1])
    annotationfile = Path(out_label_dir) / 'annotations' / (filename + '.json')


    image_dict = {"image_path": single_image_path, "image_name": filename, "annotations": [], "width": width,
                      "height": height}
    
    label_gdf = gpd.read_file(single_label_path)
    polygeos = label_gdf.geometry.to_list()
    
    for idx in range(len(polygeos)):
        # assert isinstance(polygeos[idx], shapely.geometry.polygon.Polygon)

        label_dict = {"label": label}
        regions = dict()
        
        try:
            xs, ys = polygeos[idx].exterior.coords.xy
            
            rs = list()
            cs = list()
            for x,y in zip(xs,ys):
                r, c = img.index(x, y)
                new_c, new_r = edge_pixel(c, r, width, height)
                
                rs.append(new_r)
                cs.append(new_c)
            
            regions['x'] = rs    
            regions['y'] = cs

        except:
            # only get geometry for the first polygon in multipolygon types
            coordslist = [list(i.exterior.coords.xy) for i in polygeos[idx].geoms]
            xs = coordslist[0][0].tolist()
            ys = coordslist[0][1].tolist()
            
            rs = list()
            cs = list()
            
            for x,y in zip(xs, ys):
                
                r, c = img.index(x, y)
#                 new_r = narrow_num(r, 650)
#                 new_c = narrow_num(c, 650)

                new_c, new_r = edge_pixel(c, r, width, height)
                
                rs.append(new_r)
                cs.append(new_c)
            
            regions['x'] = rs    
            regions['y'] = cs
            
        label_dict['region'] = regions
        image_dict['annotations'].append(label_dict)

        with open(annotationfile, 'w') as js:
            json.dump(image_dict, js)
            
        


In [10]:
def narrow_num(num, bound):
    
    if num >= int(bound):
        num = int(bound) - 1
    if num < 0:
        num = 0
    
    return num


def edge_pixel(col, row, cols, rows):
    
    if col == cols:
        col = cols - 10
    
    if row == rows:
        row = rows - 10
    
    return (col, row)
        


In [8]:
# Generating json annotation for each image

gen_annotation = True

while gen_annotation:
    
    gen_annotation = False

    for k in tqdm(image_dict):

        gdf = gpd.read_file(label_dict[k])
        if not gdf.empty:
            print("Working on {}".format(image_dict[k]))
            generate_annotation(image_dict[k], label_dict[k], "building")
            dest = Path(out_label_dir)/"images"
            shutil.copy(image_dict[k], dest)

  0%|          | 0/182 [00:00<?, ?it/s]

Working on /rapids/notebooks/sciclone/geograd/Miranda/github/Building_Detection_CN/DigitalGlobe_FineTune/images_wgs84/21.464444444444442_111.08194444444445_port_20210711_DT.tif


  1%|          | 2/182 [00:00<01:15,  2.37it/s]

Working on /rapids/notebooks/sciclone/geograd/Miranda/github/Building_Detection_CN/DigitalGlobe_FineTune/images_wgs84/20.050555555555555_110.14833333333334_port_20170409_DT.tif
Working on /rapids/notebooks/sciclone/geograd/Miranda/github/Building_Detection_CN/DigitalGlobe_FineTune/images_wgs84/26.57361111111111_101.63722222222223_ps_20210212_DT.tif


  2%|▏         | 3/182 [00:01<01:01,  2.90it/s]

...

 99%|█████████▉| 180/182 [01:19<00:00,  5.35it/s]

Working on /rapids/notebooks/sciclone/geograd/Miranda/github/Building_Detection_CN/DigitalGlobe_FineTune/images_wgs84/32.882777777777775_119.7363888888889_f_20201228_DT.tif
Working on /rapids/notebooks/sciclone/geograd/Miranda/github/Building_Detection_CN/DigitalGlobe_FineTune/images_wgs84/32.165277777777774_119.80027777777778_f_20171220_DT.tif


 99%|█████████▉| 181/182 [01:20<00:00,  2.40it/s]

Working on /rapids/notebooks/sciclone/geograd/Miranda/github/Building_Detection_CN/DigitalGlobe_FineTune/images_wgs84/34.64194444444444_103.17888888888889_ps_20161115_DT.tif


100%|██████████| 182/182 [01:20<00:00,  2.26it/s]


In [9]:

def split_train_val(img_folder, split_rate=None):
      
    # split_rate is the percentage of training dataset among all data
    
    rpath = os.path.dirname(img_folder)

    train_path = os.path.join(rpath, "train")
    val_path = os.path.join(rpath, "val")

    # create training and val folder
    Path(train_path).mkdir(exist_ok=True, parents=True)
    Path(val_path).mkdir(exist_ok=True, parents=True)

    img_paths = [os.path.join(img_folder, f) for f in os.listdir(img_folder) if f.endswith(".tif")]
    # define the idx of images that are used as training dataset
    nt = len(img_paths) * split_rate
    tidxs = random.sample(range(0, len(img_paths)), int(nt))
    vidxs = [i for i in range(0, len(img_paths)) if i not in tidxs]

    trains_json = list()
    vals_json = list()

    for idx in tidxs:
        basename = ".".join(os.path.basename(img_paths[idx]).split(".")[:-1])
        annotationfile = Path(out_label_dir)/ "annotations" / ('{}.json').format(basename)
        shutil.copy(img_paths[idx], train_path)
        trains_json.append(annotationfile)

    for idx in vidxs:
        basename = ".".join(os.path.basename(img_paths[idx]).split(".")[:-1])
        annotationfile = Path(out_label_dir)/ "annotations" / ('{}.json').format(basename)
        shutil.copy(img_paths[idx], val_path)
        vals_json.append(annotationfile)

    agg_annotation(trains_json, train_path)
    agg_annotation(vals_json, val_path)

def agg_annotation(annote_list, despath):

    file_dict = dict()
    desfile = os.path.join(despath, "annotation.json")
    for file in annote_list:
        f = open(file)
        img_id = ".".join(os.path.basename(file).split(".")[:-1])
        file_dict[img_id] = json.load(f)

    with open(desfile, 'w') as js:
        json.dump(file_dict, js)
        

        
# Call the code        
gen_train_val = True

while gen_train_val:
    gen_train_val = False
    split_train_val(dest, split_rate=0.8)